# Robert K Burns 
### DSC650 Assignment 5.2
### Winter Quarter 2019-2020

Creating a small data warehouse.

In [2]:
# Importing SQL package
from pyspark.sql import *

In [3]:
# Dropping any tables I may have already created in earlier rounds of testing
spark.sql('DROP TABLE IF EXISTS elem')
spark.sql('DROP TABLE IF EXISTS places')
spark.sql('DROP TABLE IF EXISTS second')
spark.sql('DROP TABLE IF EXISTS unified')

Out[7]: DataFrame[]

In [4]:
# Set warehouse directory
warehouse_dir = '/home/rkburns/spark-warehouse/'
spark = SparkSession.builder \
    .appName("DSC650Assignment5") \
    .config("spark.sql.warehouse.dir", warehouse_dir) \
    .getOrCreate()

In [5]:
# Setting file paths
csv_file_path1 = '/FileStore/tables/2017/elementary_schools.csv'
csv_file_path2 = '/FileStore/tables/2018/elementary_schools.csv'

csv_file_path3 = '/FileStore/tables/2017/places.csv'
csv_file_path4 = '/FileStore/tables/2018/places.csv'

csv_file_path5 = '/FileStore/tables/2017/secondary_schools.csv'
csv_file_path6 = '/FileStore/tables/2018/secondary_schools.csv'

csv_file_path7 = '/FileStore/tables/2017/unified_school_districts.csv'
csv_file_path8 = '/FileStore/tables/2018/unified_school_districts.csv'


In [6]:
# Load dfs and create tables (I'm sure there's a better way, I just couldn't get any to work)
df1 = spark.read.load(
  csv_file_path1,
  format='csv',
  sep=',',
  inferSchema=True,
  header=True
)

df2 = spark.read.load(
  csv_file_path2,
  format='csv',
  sep=',',
  inferSchema=True,
  header=True
)

df3 = spark.read.load(
  csv_file_path3,
  format='csv',
  sep=',',
  inferSchema=True,
  header=True
)
df4 = spark.read.load(
  csv_file_path4,
  format='csv',
  sep=',',
  inferSchema=True,
  header=True
)
df5 = spark.read.load(
  csv_file_path5,
  format='csv',
  sep=',',
  inferSchema=True,
  header=True
)
df6 = spark.read.load(
  csv_file_path6,
  format='csv',
  sep=',',
  inferSchema=True,
  header=True
)
df7 = spark.read.load(
  csv_file_path7,
  format='csv',
  sep=',',
  inferSchema=True,
  header=True
)
df8 = spark.read.load(
  csv_file_path8,
  format='csv',
  sep=',',
  inferSchema=True,
  header=True
)


elem = df1.unionAll(df2)
elem.write.saveAsTable('elem')

places = df3.unionAll(df4)
places.write.saveAsTable('places')

second = df5.unionAll(df6)
second.write.saveAsTable('second')

unified = df7.unionAll(df8)
unified.write.saveAsTable('unified')




In [7]:
# Showing the count of the combined "places" table
sqlContext.sql("SELECT COUNT(*) FROM places").show()

+--------+
count(1)|
+--------+
 59151|
+--------+

In [8]:
display(spark.sql("SELECT a.state as State, a.year as Year, a.cnt Elementary, b.cnt Secondary, c.cnt Unified \
                   FROM (SELECT state, year, count(*) as cnt \
                         FROM elementary \
                         WHERE state IN('NE','IA') \
                         GROUP BY state, year) a \
                   LEFT OUTER JOIN \
                        (SELECT state, year, count(*) as cnt \
                         FROM secondary \
                         WHERE state IN('NE','IA') \
                         GROUP BY state, year) b \
                   CROSS JOIN \
                         (SELECT state, year, count(*) as cnt \
                         FROM unified \
                         WHERE state IN('NE','IA') \
                         GROUP BY state, year) c \
                    ON a.state = b.state \
                    AND a.year = b.year \
                    ORDER BY a.state, a.year"))

State,Year,Elementary,Secondary,Unified
IA,2017,336,336,333
IA,2017,336,336,246
IA,2017,336,336,336
IA,2017,336,336,251
IA,2018,333,333,246
IA,2018,333,333,333
IA,2018,333,333,336
IA,2018,333,333,251
NE,2017,251,251,251
NE,2017,251,251,333


In [9]:
## Second part of the exercise: Recreate the report from Assignment 4.2 using SQL

In [10]:
# Dropping any tables I may have created during testing
spark.sql('DROP TABLE IF EXISTS flights')
spark.sql('DROP TABLE IF EXISTS airport_codes')

Out[14]: DataFrame[]

In [11]:
# import data
df_flights = spark.read.parquet("/FileStore/tables/flights.parquet")
df_airport_codes = spark.read.load("/FileStore/tables/airport_codes-e62ed.csv",
  format="csv",
  sep=",",
  inferSchema=True,
  header=True
)

In [12]:
df_flights.write.saveAsTable('flights')
df_airport_codes.write.saveAsTable('airport_codes')

In [13]:
display(spark.sql('''SELECT RANK() OVER (ORDER BY subtable.total_passengers DESC) as `Rank`, a.name as `Airport Name`, a.iata_code as `IATA Code`, 
            subtable.total_passengers as `Total Passengers`, INT(subtable.daily_passengers) as `Daily Passengers`, subtable.total_flights as `Total Flights`, 
            INT(subtable.daily_flights)  as `Daily Flights` FROM airport_codes a
            INNER JOIN
            (SELECT f.destination_airport_code as dest_code, SUM(f.passengers) as total_passengers, SUM(f.passengers)/365 as daily_passengers, SUM(f.flights) as total_flights, SUM(f.flights)/365 as daily_flights
            FROM flights f
            LEFT OUTER JOIN airport_codes d ON d.iata_code = f.destination_airport_code
            WHERE f.flight_year = 2008
            GROUP BY f.destination_airport_code) subtable
            ON a.iata_code = subtable.dest_code
            ORDER BY total_passengers DESC
            LIMIT 10
            '''))

Rank,Airport Name,IATA Code,Total Passengers,Daily Passengers,Total Flights,Daily Flights
1,Hartsfield Jackson Atlanta International Airport,ATL,35561795,97429,395192,1082
2,Chicago O'Hare International Airport,ORD,26398793,72325,356570,976
3,Dallas Fort Worth International Airport,DFW,22883558,62694,270243,740
4,Los Angeles International Airport,LAX,19741782,54087,215000,589
5,McCarran International Airport,LAS,18262263,50033,164123,449
6,Phoenix Sky Harbor International Airport,PHX,17305718,47412,181259,496
7,Charlotte Douglas International Airport,CLT,15038489,41201,205040,561
8,George Bush Intercontinental Houston Airport,IAH,14870717,40741,214245,586
9,Orlando International Airport,MCO,14581086,39948,131710,360
10,Detroit Metropolitan Wayne County Airport,DTW,14228887,38983,191910,525
